In [1]:
import os

In [2]:
%pwd

'/home/ehsan/PycharmProjects/Chicken-Disease-Classification/Chicken-Disease-Classification/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/ehsan/PycharmProjects/Chicken-Disease-Classification/Chicken-Disease-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_dirs

In [13]:
class ConfigurationManager:
    def __init__(
            self,
            config_dir=CONFIG_FILE_PATH,
            params_file=PARAMS_FILE_PATH):
        self.config = read_yaml(config_dir)
        self.params = read_yaml(params_file)
        
        create_dirs([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_file    
        )
        return data_ingestion_config
    

In [8]:
import os 
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [22]:
class DataIngestion:
    def __init__(self, data_config: DataIngestionConfig):
        self.config = data_config
        
    def download_data(self):
        os.makedirs(os.path.dirname(self.config.local_data_file), exist_ok=True)
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(url=self.config.source_URL,
                                                    filename=self.config.local_data_file)
            logger.info(f'Downloading {filename} to {self.config.local_data_file} with headers: {headers}'
                        )
            
        else:
            logger.info(f'File already downloaded: {self.config.local_data_file} with {get_size(Path(self.config.local_data_file))}KB size')
    
    def extract_data(self):
        
        unzip_dir = self.config.unzip_dir
        os.makedirs(unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_file:
            zip_file.extractall(unzip_dir)
            

In [23]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_data()
except Exception as e:
    raise e

2024-02-21 21:47:43,825 - INFO - common - Loaded yaml file config/config.yaml
2024-02-21 21:47:43,830 - INFO - common - Loaded yaml file params.yaml
2024-02-21 21:47:43,832 - INFO - common - created directory artifacts
2024-02-21 21:47:43,834 - INFO - 491321958 - File already downloaded: artifacts/data_ingestion/data.zip with ~ 11345 KBKB size
